In [1]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import datetime
import warnings
import pymongo
from urllib import parse

import bar_chart_race as bcr

In [2]:
# username = "zgtNewsUser"
# password = "ciiczgtJZFP"
client = pymongo.MongoClient(host="localhost", port=27017)
db = client['AvInformation']
table = db['AvInformation']

In [3]:
# 提取非论坛的数据
data = pd.DataFrame(list(table.find({"type":{ "$ne" : "BBS" }})))

In [4]:
# 提取厂商信息
def clean_publisher(text):
    try:
        text1 = re.sub('[\s+]', '', text)
        text2 = re.findall('制作商/发行商(.*)类别',text1)[0]
        text3 = re.findall('<p>(.*)</p>',text2)[0].split("$")[0]
        return text3
    except:
#         print(text)
        return "未知厂商"
data["publisher"] = data.content.apply(clean_publisher)


# 提取影片时长
def clean_duration(text):
    try:
        text1 = re.sub('[\s+]', '', text)
        text2 = re.findall('长度(.*)制作商/发行商',text1)[0]
        text3 = re.findall('</span>(.*)</p>',text2)[0].replace("分钟","")
        return text3
    except:
#         print(text)
        return "错误时间"
data["duration"] = data.content.apply(clean_duration)


# 提取番号信息
def clean_number(text):
    try:
        text1 = re.sub('[\s+]', '', text)
        text2 = re.findall('识别码(.*)发行时间',text1)[0]
        text3 = re.findall('">(.*)</span',text2)[0]
        return text3
    except:
#         print(text)
        return "错误番号"
data["number"] = data.content.apply(clean_number)


# 提取影片类型
def clean_type(text):
    try:
        text1 = re.sub('[\s+]', '', text)
        text2 = re.findall('类别(.*)',text1)[0]
        text3 = re.findall('html">(.*?)</a>',text2)
        return text3
    except:
#         print(text)
        return "错误类型"
data["type"] = data.content.apply(clean_type)


# 处理日期信息
def clean_time(text):
    try:
        time = text.replace(" ","")[:7]
        return time
    except:
        return ""
data["date"] = data.time.apply(clean_time)


# 将多名演员的数据拆分成多行
data = data.drop(['author'], axis=1).join(data['author'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('author_new'))


# 提取非错误数据
data = data[data["publisher"] != "未知厂商"]
data = data[data["duration"] != "错误时间"]
data = data[data["number"] != "错误番号"]
data = data[data["type"] != "错误类型"]

In [5]:
# # 取排名前20的厂商
# data_publisher = data.groupby('publisher').count().sort_values('_id',ascending=False)
# publisher_name = list(data_publisher.index)

# publisher_data = data[data["publisher"].isin (publisher_name)]
# publisher_data.reset_index(drop=True,inplace=True)

# # 取统计的数据
# publisher_data = publisher_data[["time","publisher",'_id']]

# publisher_pivot_table = publisher_data.pivot_table(index="time",columns='publisher',aggfunc='count',fill_value=0)
# # 重新定义列名
# publisher_pivot_table.columns = [i[1].replace("$","") for i in publisher_pivot_table.columns]

# for col in publisher_pivot_table.columns:
#     publisher_pivot_table[col] = publisher_pivot_table[col].cumsum()
    
    
# import matplotlib.pyplot as plt

# plt.rcParams['font.sans-serif']=['SimHei'] #显示中文标签
# plt.rcParams['axes.unicode_minus']=False


# # 生成GIF图像
# bcr.bar_chart_race(
#     publisher_pivot_table.tail(100), '发行商.mp4',
#     title='看看岛国哪家发行商最卖力',
#     shared_fontdict={'family':"kaiti",'color': 'rebeccapurple'},
#     n_bars=10, # 最多显示10条数据
#     steps_per_period=20,# 图像帧数
#     period_length=200, # 设置帧率，单位时间默认为500ms
#     interpolate_period=True, # 设置每帧增加标签时间
#     figsize=(16, 9), # 设置画布大小
#     dpi=200, # 设置图像分辨率
#     bar_label_size=20, # 柱状图标签文字大小
#     tick_label_size=20, # 坐标轴标签文字大小
#     title_size=30, # 标题标签文字大小
# )

In [7]:
# 取排名前1000的演员
data_author = data.groupby('author_new').count().sort_values('_id',ascending=False).head(1001)
data_author_list = list(data_author.index)
data_author_list = [i for i in data_author_list if i != '']

# 取排名前1000的演员的数据
author_data = data[data["author_new"].isin (data_author_list)]
author_data.reset_index(drop=True,inplace=True)

# 取统计的数据
author_data = author_data[["date","author_new",'_id']]

# 进行统计
author_pivot_table = author_data.pivot_table(index="date",columns='author_new',aggfunc='count',fill_value=0)

# 进行累计求和操作
for col in author_pivot_table.columns:
    author_pivot_table[col] = author_pivot_table[col].cumsum()
    
# 删除_id统计列
author_pivot_table.columns = author_pivot_table.columns.droplevel(0)
# 重置统计列名
author_pivot_table.columns = author_pivot_table.columns.tolist()


import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif']=['SimHei'] #显示中文标签
plt.rcParams['axes.unicode_minus']=False


# 生成GIF图像
bcr.bar_chart_race(
    author_pivot_table, '近20年岛国AV女优月度作品累计量排名.mp4',
    title='近20年岛国AV女优月度作品累计量排名',
#     shared_fontdict={'family':"kaiti",'color': 'rebeccapurple'},
    n_bars=10, # 最多显示10条数据
#     steps_per_period=20,# 图像帧数
#     period_length=200, # 设置帧率，单位时间默认为500ms
#     interpolate_period=True, # 设置每帧增加标签时间
#     figsize=(16, 9), # 设置画布大小
#     dpi=200, # 设置图像分辨率
#     bar_label_size=20, # 柱状图标签文字大小
#     tick_label_size=20, # 坐标轴标签文字大小
#     title_size=30, # 标题标签文字大小
)

d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 127 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 128 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 129 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 130 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 131 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 132 missing from c

d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 543 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 544 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 545 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 546 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 547 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 548 missing from c

d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 977 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 978 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 979 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 980 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 981 missing from current font.
  font.set_text(s, 0.0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:211: RuntimeWarning: Glyph 982 missing from c

d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:180: RuntimeWarning: Glyph 127 missing from current font.
  font.set_text(s, 0, flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 128 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 129 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 130 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 131 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 132 missing from

d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 357 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 358 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 359 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 360 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 361 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 362 missing f

d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 578 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 579 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 580 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 581 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 582 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 583 missing f

d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 803 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 804 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 805 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 806 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 807 missing from current font.
  font.load_char(ord(s), flags=flags)
d:\MyTools\Anaconda3_5.2.0\lib\site-packages\matplotlib\backends\backend_agg.py:176: RuntimeWarning: Glyph 808 missing f

d:\MyTools\Anaconda3_5.2.0\lib\site-packages\bar_chart_race\_make_chart.py:226: UserWarning: Some of your columns never make an appearance in the animation. To reduce color repetition, set `filter_column_colors` to `True`
  warnings.warn("Some of your columns never make an appearance in the animation. "


In [11]:
# 取统计的数据
publisher_data = data[["time","author_new",'_id']]

publisher_pivot_table = publisher_data.pivot_table(index="time",columns='author_new',aggfunc='count',fill_value=0)
# 重新定义列名
publisher_pivot_table.columns = [i[1].replace("$","") for i in publisher_pivot_table.columns]

for col in publisher_pivot_table.columns:
    publisher_pivot_table[col] = publisher_pivot_table[col].cumsum()
    
    
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif']=['SimHei'] #显示中文标签
plt.rcParams['axes.unicode_minus']=False


# 生成GIF图像
bcr.bar_chart_race(
    publisher_pivot_table, '演员.mp4',
    title='看看岛国哪位女演员最卖力',
    shared_fontdict={'family':"kaiti",'color': 'rebeccapurple'}
)

<ipython-input-11-b85b8a84404f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  publisher_data.dropna(inplace=True)
d:\MyTools\Anaconda3\lib\site-packages\bar_chart_race\_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
d:\MyTools\Anaconda3\lib\site-packages\bar_chart_race\_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))


In [ ]:
# 正则表达式测试
# data["content"][10]

# text = data["content"][10]

# text1 = re.sub('[\s+]', '', text)
# text2 = re.findall('类别(.*)',text1)[0]
# text3 = re.findall('html">(.*?)</a>',text2)

# text3

# with open("data.text",'w',encoding='utf8') as f:
#     f.write("/n".join(publisher_pivot_table.columns))